# 🎧 TheLostChapter - One Click Audio Generator

Generate audio for your bilingual books using Microsoft Edge TTS.

**No GPU needed!** Just click ▶️ on each cell.

In [ ]:
#@title 1️⃣ Setup (Run once)
!pip install -q edge-tts

import edge_tts
import asyncio
import json
import os
import re
from pathlib import Path
from IPython.display import Audio, display, HTML

print("✅ Ready!")

In [ ]:
#@title 2️⃣ Configure

#@markdown ### Book Content (paste your chapter text)
BOOK_TITLE = "Trí Nhớ Dịu Dàng" #@param {type:"string"}
CHAPTER_ID = "ch01" #@param {type:"string"}

#@markdown ### Voice Settings
VOICE = "vi-VN-HoaiMyNeural" #@param ["vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural", "en-US-AriaNeural", "en-US-GuyNeural"]
SPEED = "+0%" #@param ["-20%", "-10%", "+0%", "+10%", "+20%"]

print(f"📖 Book: {BOOK_TITLE}")
print(f"📑 Chapter: {CHAPTER_ID}")
print(f"🎤 Voice: {VOICE}")
print(f"⏱️ Speed: {SPEED}")

In [ ]:
#@title 3️⃣ Paste Your Text Here

TEXT = """
Hãy hít thở thật sâu.

Bạn có bao giờ để ý rằng những kỷ niệm đẹp nhất thường quay về khi ta không cố gắng nhớ? Chúng đến nhẹ nhàng, như làn gió mát vào buổi chiều, như mùi hương quen thuộc từ căn bếp của mẹ.

Trí nhớ không phải là một cái hộp cứng nhắc mà ta nhét đầy thông tin. Trí nhớ là một khu vườn — cần được chăm sóc, tưới nước, và quan trọng nhất, cần có thời gian để nghỉ ngơi.

Spaced Repetition — hay "Lặp lại ngắt quãng" — là một phương pháp học tập thuận theo nhịp điệu tự nhiên của trí não.

Thay vì cố nhồi nhét mọi thứ trong một đêm, ta ôn lại kiến thức vào những khoảng thời gian ngày càng dài hơn.

Mỗi lần ôn lại, ký ức trở nên sâu hơn, bền hơn — như những rễ cây ngày càng bám chặt vào đất.
""" #@param {type:"string"}

# Clean text
text = TEXT.strip()
print(f"📝 Text length: {len(text)} characters")
print(f"\n--- Preview ---\n{text[:200]}...")

In [ ]:
#@title 4️⃣ Generate Audio 🎵

OUTPUT_FILE = f"{CHAPTER_ID}.mp3"

async def generate():
    print("🔄 Generating audio...")
    communicate = edge_tts.Communicate(text, VOICE, rate=SPEED)
    await communicate.save(OUTPUT_FILE)
    print(f"✅ Saved: {OUTPUT_FILE}")

await generate()

# Play preview
display(HTML(f"<h3>🎧 Preview</h3>"))
display(Audio(OUTPUT_FILE, autoplay=False))

In [ ]:
#@title 5️⃣ Generate with Timestamps (for sentence sync)

async def generate_with_timestamps():
    print("🔄 Generating audio with timestamps...")
    communicate = edge_tts.Communicate(text, VOICE, rate=SPEED)
    
    timestamps = []
    audio_data = b""
    
    async for chunk in communicate.stream():
        if chunk["type"] == "audio":
            audio_data += chunk["data"]
        elif chunk["type"] == "WordBoundary":
            timestamps.append({
                "text": chunk["text"],
                "start": round(chunk["offset"] / 10_000_000, 3),
                "end": round((chunk["offset"] + chunk["duration"]) / 10_000_000, 3)
            })
    
    # Save audio
    with open(OUTPUT_FILE, "wb") as f:
        f.write(audio_data)
    
    # Save timestamps
    ts_file = OUTPUT_FILE.replace(".mp3", "_timestamps.json")
    with open(ts_file, "w", encoding="utf-8") as f:
        json.dump(timestamps, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Audio: {OUTPUT_FILE}")
    print(f"✅ Timestamps: {ts_file} ({len(timestamps)} words)")
    return timestamps

timestamps = await generate_with_timestamps()

# Show sample
print("\n--- Sample timestamps ---")
for ts in timestamps[:5]:
    print(f"  {ts['start']:.2f}s - {ts['end']:.2f}s: {ts['text']}")

In [ ]:
#@title 6️⃣ Download Files

from google.colab import files

# Download audio
files.download(OUTPUT_FILE)

# Download timestamps if exists
ts_file = OUTPUT_FILE.replace(".mp3", "_timestamps.json")
if os.path.exists(ts_file):
    files.download(ts_file)

print("📥 Download started!")

---

## 🚀 Batch Mode: Generate from GitHub

If your content is already in the repo, run the cell below:

In [ ]:
#@title 🔄 Clone Repo & Generate All Chapters

GITHUB_REPO = "https://github.com/nmnhut-it/english-learning-app" #@param {type:"string"}
BOOK_ID = "gentle-mind" #@param {type:"string"}
LANGUAGE = "vi" #@param ["vi", "en"]

# Clone repo
!git clone --depth 1 {GITHUB_REPO} repo 2>/dev/null || (cd repo && git pull)

# Run generator
%cd repo/the-lost-chapter/tools/tts
!python generate_audio.py --book {BOOK_ID} --lang {LANGUAGE}

print("\n✅ All chapters generated!")
print("📁 Files in: repo/the-lost-chapter/content/books/{BOOK_ID}/audio/")

In [ ]:
#@title 📥 Download All Generated Audio

import shutil
from google.colab import files

# Zip audio folder
audio_dir = f"../../content/books/{BOOK_ID}/audio"
if os.path.exists(audio_dir):
    shutil.make_archive("audio_files", "zip", audio_dir)
    files.download("audio_files.zip")
    print("📥 Downloading audio_files.zip")
else:
    print("❌ No audio files found. Run generation first.")